# Section3 Project Colab
* 코랩에서 모델 피팅을 담당하는부분
* 로컬환경의 GPU보다 코랩프로의 a100이 성능이 훨씬좋기때문에  
피팅은 전부 코랩에서 진행


In [1]:
try:
    import google.colab
    google.colab.drive.mount('/content/drive')
    path = "/content/drive/MyDrive/AI_DB_gdrive/project3_data/to_colab_data/"
except:
    path = "to_colab_data/"
    pass

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow.keras as keras

import os

# 데이터셋 불러오기
train_ds = tf.data.Dataset.load(path+'train',
                               compression='None')
valid_ds = tf.data.Dataset.load(path+'valid',
                               compression='None')
test_ds = tf.data.Dataset.load(path+'test',
                               compression='None')

# 데이터 준비
train_ds = train_ds.cache()
train_ds = train_ds.batch(32).prefetch(tf.data.AUTOTUNE)
valid_ds = valid_ds.cache().batch(32)
test_df = test_ds.batch(32)


In [3]:
# 콜백 정의
# reduce_lr : 학습이 잘 안되면(3 epochs) 학습률 감소
# e_stop : 특정epoch이상 차도 없으면(10 epochs) 중단
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                                 factor=0.5, patience=3,
                                                 verbose=0, mode='auto')
e_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                          patience=10,
                                          restore_best_weights=True)
# TensorBoard callback 정의
TBoard1 = keras.callbacks.TensorBoard(
    log_dir='/content/Tensorboard/resnet',
    histogram_freq=1
)

In [4]:
# Custom ResNet 학습
model = keras.models.load_model(path+'Custom_ResNet.keras')
model.compile(optimizer=keras.optimizers.Adam(0.01),
              loss='sparse_categorical_crossentropy',
              metrics=['acc'])

hist = model.fit(train_ds,
                 validation_data=valid_ds,
                 epochs=100,
                 callbacks=[reduce_lr, e_stop, TBoard1])

model._name = 'Trained_ResNet'
keras.models.save_model(model, '/content/Trained_ResNet.keras')
hist_csv = pd.DataFrame(hist.history)
hist_csv.to_csv('ResNetHist.csv')
# 약 50epoch 에서 훈련중단되었음.

Epoch 1/100
148/148 [==============================] - 65s 278ms/step - loss: 0.8325 - acc: 0.6636 - val_loss: 16.2977 - val_acc: 0.2935 - lr: 0.0100
Epoch 2/100
148/148 [==============================] - 5s 36ms/step - loss: 0.5332 - acc: 0.7630 - val_loss: 6.2774 - val_acc: 0.2957 - lr: 0.0100
Epoch 3/100
148/148 [==============================] - 5s 36ms/step - loss: 0.4642 - acc: 0.7981 - val_loss: 17.9198 - val_acc: 0.4891 - lr: 0.0100
Epoch 4/100
148/148 [==============================] - 5s 36ms/step - loss: 0.4284 - acc: 0.8155 - val_loss: 13.9176 - val_acc: 0.3891 - lr: 0.0100
Epoch 5/100
148/148 [==============================] - 5s 36ms/step - loss: 0.4072 - acc: 0.8305 - val_loss: 2.6911 - val_acc: 0.3761 - lr: 0.0100
Epoch 6/100
148/148 [==============================] - 5s 36ms/step - loss: 0.4106 - acc: 0.8271 - val_loss: 38.4544 - val_acc: 0.3543 - lr: 0.0100
Epoch 7/100
148/148 [==============================] - 5s 36ms/step - loss: 0.4197 - acc: 0.8250 - val_loss: 6.5

In [5]:
# Custom ResNetV2 콜백 정의
# reduce_lr : 학습이 잘 안되면(3 epochs) 학습률 감소
# e_stop : 특정epoch이상 차도 없으면(20 epochs) 중단
reduce_lr2 = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                                 factor=0.5, patience=3,
                                                 verbose=0, mode='auto')
e_stop2 = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                           patience=20,
                                           restore_best_weights=True)
TBoard2 = keras.callbacks.TensorBoard(
    log_dir='/content/Tensorboard/resnetv2',
    histogram_freq=1
)

In [6]:
# Custom ResNetV2 학습
model2 = keras.models.load_model(path+'Custom_ResNetV2.keras')
model2.compile(optimizer=keras.optimizers.Adam(0.01),
               loss='sparse_categorical_crossentropy',
               metrics=['acc'])

hist2 = model2.fit(train_ds,
                   validation_data=valid_ds,
                   epochs=200,
                   callbacks=[reduce_lr2, e_stop2, TBoard2])

model2._name = 'Trained_ResNetV2'
keras.models.save_model(model2, '/content/Trained_ResNetV2.keras')
hist_csv = pd.DataFrame(hist2.history)
hist_csv.to_csv('ResNetV2Hist.csv')

Epoch 1/200
148/148 [==============================] - 29s 65ms/step - loss: 0.6453 - acc: 0.7158 - val_loss: 2.1111 - val_acc: 0.3457 - lr: 0.0100
Epoch 2/200
148/148 [==============================] - 8s 54ms/step - loss: 0.4345 - acc: 0.8168 - val_loss: 2.3175 - val_acc: 0.3457 - lr: 0.0100
Epoch 3/200
148/148 [==============================] - 8s 54ms/step - loss: 0.3622 - acc: 0.8506 - val_loss: 1.3159 - val_acc: 0.5174 - lr: 0.0100
Epoch 4/200
148/148 [==============================] - 8s 54ms/step - loss: 0.2969 - acc: 0.8804 - val_loss: 1.4722 - val_acc: 0.4739 - lr: 0.0100
Epoch 5/200
148/148 [==============================] - 8s 54ms/step - loss: 0.2949 - acc: 0.8798 - val_loss: 2.3588 - val_acc: 0.5239 - lr: 0.0100
Epoch 6/200
148/148 [==============================] - 8s 54ms/step - loss: 0.2756 - acc: 0.8866 - val_loss: 2.7501 - val_acc: 0.4674 - lr: 0.0100
Epoch 7/200
148/148 [==============================] - 8s 54ms/step - loss: 0.2088 - acc: 0.9192 - val_loss: 0.2381 -

In [7]:
# 모델 검증성능 평가

_, acc1 = model.evaluate(valid_ds)
_, acc2 = model2.evaluate(valid_ds)

print(f"ResNet   valid acc : {round(acc1,3)}")
print(f"ResNetV2 valid acc : {round(acc2,3)}")

15/15 [==============================] - 0s 19ms/step - loss: 0.0333 - acc: 0.9957
ResNet   valid acc : 0.985
ResNetV2 valid acc : 0.996


In [8]:
!zip -r /content/TBoard.zip /content/Tensorboard

  adding: content/Tensorboard/ (stored 0%)
  adding: content/Tensorboard/resnet/ (stored 0%)
  adding: content/Tensorboard/resnet/train/ (stored 0%)
  adding: content/Tensorboard/resnet/train/events.out.tfevents.1684066141.a44648d41998.834.0.v2 (deflated 69%)
  adding: content/Tensorboard/resnet/validation/ (stored 0%)
  adding: content/Tensorboard/resnet/validation/events.out.tfevents.1684066201.a44648d41998.834.1.v2 (deflated 77%)
  adding: content/Tensorboard/resnetv2/ (stored 0%)
  adding: content/Tensorboard/resnetv2/train/ (stored 0%)
  adding: content/Tensorboard/resnetv2/train/events.out.tfevents.1684066500.a44648d41998.834.2.v2 (deflated 69%)
  adding: content/Tensorboard/resnetv2/validation/ (stored 0%)
  adding: content/Tensorboard/resnetv2/validation/events.out.tfevents.1684066527.a44648d41998.834.3.v2 (deflated 78%)
